<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/wildfires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#read in the csv file generated from my SQL query of the database into a pandas dataframe
#The dataset comes from a SQLite database found on Kaggle: https://www.kaggle.com/datasets/rtatman/188-million-us-wildfires
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
wildfires = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/CA_wildfires_since2000.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
wildfires.head(10)

,OBJECTID,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,FIRE_NAME,FIRE_YEAR,DISCOVERY_DOY,CONT_DOY,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_NAME
0,1,USCAPNF,Plumas National Forest,0511,Plumas National Forest,FOUNTAIN,2005,33,33.0,Miscellaneous,0.10,A,40.036944,-121.005833,USFS,CA,63,Plumas
1,2,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,PIGEON,2004,133,133.0,Lightning,0.25,A,38.933056,-120.404444,USFS,CA,61,Placer
2,3,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,SLACK,2004,152,152.0,Debris Burning,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17,El Dorado
3,4,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,DEER,2004,180,185.0,Lightning,0.10,A,38.559167,-119.913333,USFS,CA,3,Alpine
4,5,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,STEVENOT,2004,180,185.0,Lightning,0.10,A,38.559167,-119.933056,USFS,CA,3,Alpine
5,6,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,HIDDEN,2004,182,183.0,Lightning,0.10,A,38.635278,-120.103611,USFS,CA,5,Amador
6,7,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,FORK,2004,183,184.0,Lightning,0.10,A,38.688333,-120.153333,USFS,CA,17,El Dorado
7,8,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,SLATE,2005,67,67.0,Debris Burning,0.80,B,40.968056,-122.433889,STATE OR PRIVATE,CA,NaN,NaN
8,9,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,SHASTA,2005,74,74.0,Debris Burning,1.00,B,41.233611,-122.283333,STATE OR PRIVATE,CA,NaN,NaN
9,10,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,TANGLEFOOT,2004,183,184.0,Lightning,0.10,A,38.548333,-120.149167,USFS,CA,5,Amador


In [47]:
#Print the shape of the data: Here we see we have data on over 100,000
#California wildfires since 2000
wildfires.shape

(114558, 18)

In [48]:
wildfires.drop(['OBJECTID', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'NWCG_REPORTING_UNIT_NAME'], axis = 1, inplace = True)

In [49]:
#Use the pandas info function to get an overview of the different features within the dataset and their datatypes as well as NaN counts
wildfires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114558 entries, 0 to 114557
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   SOURCE_REPORTING_UNIT_NAME  114558 non-null  object 
 1   FIRE_NAME                   103308 non-null  object 
 2   FIRE_YEAR                   114558 non-null  int64  
 3   DISCOVERY_DOY               114558 non-null  int64  
 4   CONT_DOY                    61574 non-null   float64
 5   STAT_CAUSE_DESCR            114558 non-null  object 
 6   FIRE_SIZE                   114558 non-null  float64
 7   FIRE_SIZE_CLASS             114558 non-null  object 
 8   LATITUDE                    114558 non-null  float64
 9   LONGITUDE                   114558 non-null  float64
 10  OWNER_DESCR                 114558 non-null  object 
 11  STATE                       114558 non-null  object 
 12  COUNTY                      51834 non-null   object 
 13  FIPS_NAME     

In [50]:
#Take not of a few things here:
#The average wildfire size in this dataset is 81 acres, most fires last around 2.5 days
#Another takeaway: this database only contains fires from 2001 - 2015. We will join it 
#with a second dataset that has fires from 2013-2020 also from Kaggle
#(https://www.kaggle.com/datasets/ananthu017/california-wildfire-incidents-20132020)

wildfires.describe()

,FIRE_YEAR,DISCOVERY_DOY,CONT_DOY,FIRE_SIZE,LATITUDE,LONGITUDE
count,114558.000000,114558.000000,61574.000000,114558.000000,114558.000000,114558.000000
mean,2007.922380,197.569450,200.625231,80.792444,37.495814,-120.259658
std,4.293784,68.558497,67.587688,2446.430633,2.551040,2.129471
min,2001.000000,1.000000,1.000000,0.001000,32.544965,-124.402883
25%,2004.000000,159.000000,164.000000,0.100000,35.425000,-121.770258
50%,2008.000000,198.000000,203.000000,0.250000,37.697311,-120.596944
75%,2012.000000,244.000000,245.000000,1.000000,39.458972,-118.754703
max,2015.000000,366.000000,366.000000,315578.800000,42.038090,-114.155500


In [51]:
wildfires2 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/California_Fire_Incidents.csv', usecols = [0,2,4,9,10,14,21,23,25,31])

In [52]:
wildfires2.head()

,AcresBurned,AdminUnit,ArchiveYear,Counties,CountyIds,Extinguished,Latitude,Longitude,Name,Started
0,257314.0,Stanislaus National Forest/Yosemite National Park,2013,Tuolumne,55,2013-09-06T18:30:00Z,37.857000,-120.086000,Rim Fire,2013-08-17T15:25:00Z
1,30274.0,USFS Angeles National Forest/Los Angeles Count...,2013,Los Angeles,19,2013-06-08T18:30:00Z,34.585595,-118.423176,Powerhouse Fire,2013-05-30T15:28:00Z
2,27531.0,CAL FIRE Riverside Unit / San Bernardino Natio...,2013,Riverside,33,2013-07-30T18:00:00Z,33.709500,-116.728850,Mountain Fire,2013-07-15T13:43:00Z
3,27440.0,Tahoe National Forest,2013,Placer,31,2013-08-30T08:00:00Z,39.120000,-120.650000,American Fire,2013-08-10T16:30:00Z
4,24251.0,Ventura County Fire/CAL FIRE,2013,Ventura,56,2013-05-11T06:30:00Z,0.000000,0.000000,Springs Fire,2013-05-02T07:01:00Z


In [77]:
wildfires.rename(columns = {'FIRE_SIZE':'AcresBurned', 'FIRE_YEAR':'Year', 'FIRE_NAME':'Name', 'DISCOVERY_DOY':'Discovered DOY'
                  ,'CONT_DOY':'Contained DOY', 'STAT_CAUSE_DESCR':'Cause','LATITUDE':'Latitude', 'LONGITUDE': 'Logitude', 'COUNTY':'CountyIds',
                  'STATE':'State','FIRE_SIZE_CLASS': 'Fire Size Rank', 'FIPS_NAME': 'County'
                  }, inplace = True)
wildfires2.rename(columns = {'ArchiveYear':'Year', 
                             'Started':'Discovered DOY', 'Extinguished':'Contained DOY', 'Counties': 'County'},
                  inplace = True)

In [78]:
wildfires.head(20)

,SOURCE_REPORTING_UNIT_NAME,Name,Year,Discovered DOY,Contained DOY,Cause,AcresBurned,Fire Size Rank,Latitude,Logitude,OWNER_DESCR,State,CountyIds,County,ZIP
0,Plumas National Forest,FOUNTAIN,2005,33,33.0,Miscellaneous,0.10,A,40.036944,-121.005833,USFS,CA,63,Plumas,95984
1,Eldorado National Forest,PIGEON,2004,133,133.0,Lightning,0.25,A,38.933056,-120.404444,USFS,CA,61,Placer,NaN
2,Eldorado National Forest,SLACK,2004,152,152.0,Debris Burning,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17,El Dorado,NaN
3,Eldorado National Forest,DEER,2004,180,185.0,Lightning,0.10,A,38.559167,-119.913333,USFS,CA,3,Alpine,NaN
4,Eldorado National Forest,STEVENOT,2004,180,185.0,Lightning,0.10,A,38.559167,-119.933056,USFS,CA,3,Alpine,NaN
5,Eldorado National Forest,HIDDEN,2004,182,183.0,Lightning,0.10,A,38.635278,-120.103611,USFS,CA,5,Amador,NaN
6,Eldorado National Forest,FORK,2004,183,184.0,Lightning,0.10,A,38.688333,-120.153333,USFS,CA,17,El Dorado,NaN
7,Shasta-Trinity National Forest,SLATE,2005,67,67.0,Debris Burning,0.80,B,40.968056,-122.433889,STATE OR PRIVATE,CA,NaN,NaN,96051
8,Shasta-Trinity National Forest,SHASTA,2005,74,74.0,Debris Burning,1.00,B,41.233611,-122.283333,STATE OR PRIVATE,CA,NaN,NaN,96025
9,Eldorado National Forest,TANGLEFOOT,2004,183,184.0,Lightning,0.10,A,38.548333,-120.149167,USFS,CA,5,Amador,NaN


In [79]:
wildfires2.head(20)
#We need to add features fire size class, discovery DOY and contained DOY to the wildfires

,AcresBurned,AdminUnit,Year,County,CountyIds,Contained DOY,LATITUDE,LONGITUDE,Name,Discovered DOY
0,257314.0,Stanislaus National Forest/Yosemite National Park,2013,Tuolumne,55,2013-09-06T18:30:00Z,37.857000,-120.086000,Rim Fire,2013-08-17T15:25:00Z
1,30274.0,USFS Angeles National Forest/Los Angeles Count...,2013,Los Angeles,19,2013-06-08T18:30:00Z,34.585595,-118.423176,Powerhouse Fire,2013-05-30T15:28:00Z
2,27531.0,CAL FIRE Riverside Unit / San Bernardino Natio...,2013,Riverside,33,2013-07-30T18:00:00Z,33.709500,-116.728850,Mountain Fire,2013-07-15T13:43:00Z
3,27440.0,Tahoe National Forest,2013,Placer,31,2013-08-30T08:00:00Z,39.120000,-120.650000,American Fire,2013-08-10T16:30:00Z
4,24251.0,Ventura County Fire/CAL FIRE,2013,Ventura,56,2013-05-11T06:30:00Z,0.000000,0.000000,Springs Fire,2013-05-02T07:01:00Z
5,22992.0,Sierra National Forest,2013,Fresno,10,2013-09-24T20:15:00Z,37.279000,-119.318000,Aspen Fire,2013-07-22T22:15:00Z
6,20292.0,CAL FIRE Riverside Unit / San Bernardino Natio...,2013,Riverside,33,2013-08-12T18:00:00Z,33.861570,-116.904270,Silver Fire,2013-08-07T14:05:00Z
7,14754.0,Klamath National Forest,2013,Siskiyou,47,2013-08-31T06:45:00Z,41.320000,-123.176000,Salmon River Complex,2013-07-31T22:00:00Z
8,12503.0,Six Rivers National Forest,2013,Humboldt,12,2013-08-12T12:00:00Z,41.035000,-123.488000,Corral Complex,2013-08-10T11:40:00Z
9,11429.0,CAL FIRE Tehama-Glenn Unit,2013,Tehama,52,2013-08-29T16:45:00Z,40.042630,-121.853970,Deer Fire,2013-08-23T14:15:00Z


In [54]:
wildfires2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AcresBurned   1633 non-null   float64
 1   AdminUnit     1636 non-null   object 
 2   Year          1636 non-null   int64  
 3   Counties      1636 non-null   object 
 4   CountyIds     1636 non-null   object 
 5   Extinguished  1577 non-null   object 
 6   LATITUDE      1636 non-null   float64
 7   LONGITUDE     1636 non-null   float64
 8   Name          1636 non-null   object 
 9   Started       1636 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 127.9+ KB


In [55]:
#Next steps
#1. drop NaN values
#2. Map longtitude and latitude to zip code of the wildfire
#3. Add in NOAA weather features

In [56]:
#Feature Engineering

#zipcode - get from 
#Population of zip code - does presence humans affect wildfires?
#information on snowfall

#month began

#monthly percipitation

#average temperatures
#days exceeding 90

#Some kind of way to determine if an area has already burned. A largely burned area is less likely to burn in future years
#build a function to see if a wildfire was in the area within the past x amount of years (use latitude and longitude w acres burned)
#rec areae


Adding Additional Features to the Dataset

In [57]:
#Write a function that gives fire's US zip code given coordinates in longitude and latitude
#Help from stack overflow: https://gis.stackexchange.com/questions/352961/convert-lat-lon-to-zip-postal-code-using-python
import geopy as gp
from geopy import Nominatim

def coordinates_to_zipcode(lat, long, geolocater):
  location = geolocater.reverse((lat, long))
  if 'postcode' in location.raw['address'].keys():
    #print(location)
    return location.raw['address']['postcode']
  else:
    #print('None')
    return 'NaN'

geolocater = gp.Nominatim(user_agent='my-application')
print(coordinates_to_zipcode(40.036944, -121.005833, geolocater))

95984


In [58]:
wildfires['ZIP'] = ''
for index, row in wildfires.iterrows():
  wildfires.at[index,'ZIP'] = coordinates_to_zipcode(wildfires.at[index,'LATITUDE'], wildfires.at[index,'LONGITUDE'], geolocater)

KeyboardInterrupt: ignored

In [ ]:
print(wildfires['ZIP'])